In [34]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB6
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.utils import class_weight

In [35]:
model = EfficientNetB6(include_top=True)

174465024/174460376 [==============================] - 15s 0us/step

In [39]:
def get_features(ID):
    img_path = 'food_porn/food/' + ID + '.jpg'
    img = image.load_img(img_path, target_size=(528, 528,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = np.array(model.predict(x))
    return features

In [41]:
data = np.zeros((10000,1001))
id = np.array(range(0,10000))
data[:,0] = id.transpose()

def features():
    for i in range(0,10000):
        featureid = '{0:05}'.format(i)
        features = get_features(featureid)
        data[i,1:] = features
        if i % 100 == True:
            print(i)
    return data

data = features()

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
9001
9101
9201
9301
9401
9501
9601
9701
9801
9901


In [42]:
print(np.count_nonzero(data))


np.savetxt("features_effb6.csv", data, delimiter=',')

10009999


using false we got:     527 correct
using true we got:      520 correct

how about... we use the true version to feed a NN

In [67]:
train_trip = np.loadtxt("train_triplets.txt").astype('int')

data = np.genfromtxt("features_xception.csv", delimiter= ',')
data = np.array(data[:,1:])

def load_features(int_id):
    return data[int_id]

In [68]:
%%time

#create the base dataset with the 2 sets of labels
data_y1 = np.ones((59515,1))
data_y0 = np.zeros((59515,1))

#create the inverse dataset:
train_trip_inv = np.zeros((59515,3)).astype('int')
train_trip_inv[:,0] = train_trip[:,0]
train_trip_inv[:,1] = train_trip[:,2]
train_trip_inv[:,2] = train_trip[:,1]

#assemble everything into one huge pile of S*** in order to shuffle it
id_fin = np.concatenate((train_trip,train_trip_inv),axis=0)
labels = np.concatenate((data_y1,data_y0),axis=0)
id_labels = np.concatenate((id_fin,labels),axis=1)

data_nn_x = np.zeros((119030,3000))

for i in range(0,3):
        for j in range(0,119030):
            data_nn_x[j,range(i*1000,1000*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)


#print(train_trip)
#print(train_trip_inv)
#now we try to randomize the data in order to increase our accuracy
# we may even double the dataset by using 0 as well as the 1 case

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 0 60000
we're at entry: 0 70000
we're at entry: 0 80000
we're at entry: 0 90000
we're at entry: 0 100000
we're at entry: 0 110000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 1 60000
we're at entry: 1 70000
we're at entry: 1 80000
we're at entry: 1 90000
we're at entry: 1 100000
we're at entry: 1 110000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000
we're at entry: 2 60000
we're at entry: 2 70000
we're at entry: 2 80000
we're at entry: 2 90000
we're at entry: 2 100000
we're at entry: 2 110000
Wall time: 29.8 s


In [69]:
collab = np.concatenate((data_nn_x,labels), axis=1)
np.random.shuffle(collab)

data_x = collab[:,:3000]
data_y = collab[:,3000]

In [ ]:
#print("%d bytes" % (data_nn_x.size * data_nn_x.itemsize))
#np.savetxt("nn_data.csv",data_nn_x,delimiter=',')

In [70]:
%%time
#TAKE IT Branda:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(1024, activation='relu', input_dim=3000))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(512, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(64, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())

neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(data_x, data_y, epochs=128, batch_size=1024, verbose=2, validation_split = 0.2)

#evaluation of the network prediction
#predict_test_nn = neuralNetwork.predict(df_test_onehot_nn)
#predict_train_nn = neuralNetwork.predict(df_train_onehot_nn)
#print(predict_test_nn)
#print(predict_train_nn)
#predict_test_nn = (predict_test_nn >= 0.5)
neuralNetwork.summary()

Epoch 1/128
93/93 - 2s - loss: 0.7628 - accuracy: 0.5033 - val_loss: 0.6936 - val_accuracy: 0.4989
Epoch 2/128
93/93 - 1s - loss: 0.7182 - accuracy: 0.5102 - val_loss: 0.6935 - val_accuracy: 0.4929
Epoch 3/128
93/93 - 1s - loss: 0.7059 - accuracy: 0.5158 - val_loss: 0.6933 - val_accuracy: 0.4987
Epoch 4/128
93/93 - 1s - loss: 0.6998 - accuracy: 0.5168 - val_loss: 0.6929 - val_accuracy: 0.5085
Epoch 5/128
93/93 - 1s - loss: 0.6950 - accuracy: 0.5224 - val_loss: 0.6923 - val_accuracy: 0.5170
Epoch 6/128
93/93 - 1s - loss: 0.6914 - accuracy: 0.5286 - val_loss: 0.6903 - val_accuracy: 0.5210
Epoch 7/128
93/93 - 1s - loss: 0.6879 - accuracy: 0.5353 - val_loss: 0.6880 - val_accuracy: 0.5306
Epoch 8/128
93/93 - 1s - loss: 0.6843 - accuracy: 0.5422 - val_loss: 0.6878 - val_accuracy: 0.5312
Epoch 9/128
93/93 - 1s - loss: 0.6801 - accuracy: 0.5483 - val_loss: 0.6850 - val_accuracy: 0.5372
Epoch 10/128
93/93 - 1s - loss: 0.6750 - accuracy: 0.5548 - val_loss: 0.6874 - val_accuracy: 0.5353
Epoch 11/

In [71]:
test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((59544,3000))
for i in range(0,3):
        for j in range(0,59544):
            test_nn_x[j,range(i*1000,1000*(i+1))] = load_features(test_trip[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

predict_test_nn = neuralNetwork.predict(test_nn_x)

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000


In [72]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission_xcep.txt",np.round(export,decimals=0), fmt='%i')